In [1]:


import os
import os.path as osp
from typing import Dict, List, Tuple, Union, Callable
import json
import logging

from pathlib import Path

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from tqdm import tqdm
tqdm.pandas()

import seaborn as sns

from pandarallel import pandarallel
from flair.data import Sentence

import transformers

from swann.data import ColeridgeDataset
from swann.utils import seed_everything, PipelineConfig
from swann.data.sampling import DataSplitter



/opt/anaconda3/venvs/PY38_MAIN/lib/python3.8/site-packages/tqdm/std.py:670: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
/opt/anaconda3/venvs/PY38_MAIN/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:
BASE_PATH = '.'
TRAIN_BASE_PATH = osp.join(BASE_PATH, 'datasets/train/')
TEST_BASE_PATH  = osp.join(BASE_PATH, 'datasets/test/')
SUB_FILEPATH = osp.join(BASE_PATH, 'datasets/sample_submission.csv')
TRAIN_FILEPATH = osp.join(BASE_PATH, 'datasets/train.csv')

LOGGING_FILEPATH = BASE_PATH + 'logs/'

logging.basicConfig(#filename=osp.join(LOGGING_FILEPATH, 'log.log'), 
level=logging.INFO, 
format='%(asctime)s | %(name)s | %(levelname)s | %(message)s')


COLS = {
    "id": "Id",
    "pub_title": "pub_title",
    "dataset_title": "dataset_title",
    "dataset_label": "dataset_label",
    "cleaned_label": "cleaned_label",
    "section_text": "text"
}

In [3]:
from swann.data.flair import FlairSectionColeridgeDataset

In [4]:
NB_WORKERS = 4

TOKENIZER = "flair"

CONFIG = """

splits:
    train: 0.1
    valid: 0.1
    test: 0.1
"""

config = PipelineConfig(CONFIG)

/home/lbony/projects/coleridge_challenge/coleridge_challenge/swann/utils/config.py:19: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  self.config = yaml.load(config)


### Inputs

In [5]:
train_df: pd.DataFrame = pd.read_csv(TRAIN_FILEPATH, nrows=1_000)

In [6]:
seed_everything()
pandarallel.initialize(nb_workers=NB_WORKERS, progress_bar=True)

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


### Preprocessing

In [7]:


# Split datasets
splitter = DataSplitter([config["splits"][split_name] for split_name in config["splits"]])
train_df, valid_df, test_df = splitter.split(train_df[:10_000])



2021-04-21 23:53:52,725 | root | INFO | Processed ratio 0.1 nb indices: 100
2021-04-21 23:53:52,734 | root | INFO | Processed ratio 0.1 nb indices: 100
2021-04-21 23:53:52,736 | root | INFO | Processed ratio 0.1 nb indices: 100


In [9]:
train = FlairSectionColeridgeDataset(train_df, TRAIN_BASE_PATH, COLS)
train.index_labels()
train.make_flair_ner_dataset('train.txt')

100it [00:00, 436.31it/s]


In [12]:
from datasets import load_dataset

dataset = load_dataset('csv', '/home/lbony/tmp/train.txt', parse_options)

InvalidConfigName: Bad characters from black list '<>:/\|?*' found in '/home/lbony/tmp/train.txt'. They could create issues when creating a directory for this config on Windows filesystem.

In [30]:
# Create and indexing datasets
train = ColeridgeDataset(train_df, TRAIN_BASE_PATH, COLS)
valid = ColeridgeDataset(valid_df, TRAIN_BASE_PATH, COLS)
test = ColeridgeDataset(test_df, TRAIN_BASE_PATH, COLS)

logging.info("Indexing labels on train")
train.index_labels()
logging.info("Indexing labels on valid")
valid.index_labels()
logging.info("Indexing labels on test")
test.index_labels()

2021-03-31 19:12:22,049 | root | INFO | Indexing labels on train
100it [00:00, 1097.38it/s]
2021-03-31 19:12:22,143 | root | INFO | Indexing labels on valid
100it [00:00, 1105.02it/s]
2021-03-31 19:12:22,237 | root | INFO | Indexing labels on test
100it [00:00, 1250.78it/s]


In [31]:
logging.info("Making train flair ner dataset")
train.make_flair_ner_dataset('train.txt')
logging.info("Making valid flair ner dataset")
valid.make_flair_ner_dataset('valid.txt')
logging.info("Making test flair ner dataset")
test.make_flair_ner_dataset('test.txt')

2021-03-31 19:12:28,280 | root | INFO | Making train flair ner dataset


SystemError: Python version should be 3.{5, 6, 7, 8}